In [ ]:
import ell
import subprocess
import tempfile
import os
import datetime
import uuid
from typing import Callable, Dict, List, Optional
from enum import Enum
from pydantic import BaseModel, Field
from colorama import Fore, Style, init
from tabulate import tabulate
init()

# Import classes from code_db.py
from code_db import TestStatusEnum, TestResult, Modification, UnitTest, Function, CodeDatabase

In [2]:
# Enums for Test Status
from enum import Enum

In [15]:
class JuliaCodePackage(BaseModel):
    code: str = Field(description="The function.")
    tests: str = Field(description="A single test case for the function that uses @assert.")
    test_name: str = Field(description="A concise test name in snake_case.")
    test_description: str = Field(description="A concise test description.")
    input_types: str = Field(description="The input types for the function.")
    return_types: str = Field(description="The expected output types for the function.")
    short_description: str = Field(description="A short description of the function.")
    function_name: str = Field(description="The name of the function.")

In [16]:
# Step 1: Generate a Julia function based on a description
@ell.complex(model="gpt-4o", response_format=JuliaCodePackage)
def generate_julia_function(description: str):
    """You are a Julia programmer, and you need to generate a function based on a description and a SINGLE test case."""
    prompt = f"Write a Julia function to {description}"
    # The decorator is assumed to send this prompt to GPT-4 and return the generated code
    return prompt

@ell.complex(model="gpt-4o", response_format=JuliaCodePackage)
def modify_julia_function(description: str, function_code: str):
    """You are a Julia programmer, and you need to modify a function based on the description. Maintain the existing function signature."""
    prompt = f"Modify the BODY of the Julia function {function_code} based on the following description: {description}"
        
    # The decorator is assumed to send this prompt to GPT-4 and return the generated code
    return prompt

# Step 2: Test the generated function
@ell.simple(model="gpt-4o")
def write_test_case(function_name: str) -> str:
    """Write a test case for the function."""
    prompt = f"Write a Julia test case to verify the `{function_name}` function"
    return prompt

# Step 3: Evaluate the output of the function
@ell.simple(model="gpt-4o")
def evaluate_output(expected_output: str, actual_output: str) -> str:
    """Evaluate whether the output matches the expectation."""
    prompt = f"Does the actual output `{actual_output}` match the expected `{expected_output}`?"
    return prompt

In [17]:
def pretty_print_function(code_db: CodeDatabase, function_id: str):
    func = code_db.functions.get(function_id)
    if not func:
        print(f"{Fore.RED}Function ID {function_id} not found.{Style.RESET_ALL}")
        return

    print(f"{Fore.CYAN}\n\nFunction ID: {func.function_id}{Style.RESET_ALL}")
    print(f"{Fore.CYAN}Name: {func.name}{Style.RESET_ALL}")
    print(f"{Fore.CYAN}Description: {func.description}{Style.RESET_ALL}")
    print(f"{Fore.CYAN}Creation Date: {func.creation_date.isoformat()}{Style.RESET_ALL}")
    print(f"{Fore.CYAN}Last Modified Date: {func.last_modified_date.isoformat()}{Style.RESET_ALL}")
    print(f"{Fore.CYAN}\nCode Snippet:\n ```julia\n {func.code_snippet}{Style.RESET_ALL}```")

    print(f"{Fore.YELLOW}\nUnit Tests:{Style.RESET_ALL}")
    for test in func.unit_tests:
        print(f"{Fore.GREEN}Test ID: {test.test_id}{Style.RESET_ALL}")
        print(f"{Fore.GREEN}Name: {test.name}{Style.RESET_ALL}")
        print(f"{Fore.GREEN}Description: {test.description}{Style.RESET_ALL}")
        print(f"{Fore.GREEN}Test Case:\n{test.test_case}{Style.RESET_ALL}")
        print()

    print(f"{Fore.YELLOW}Modifications:{Style.RESET_ALL}")
    for mod in code_db.modifications:
        if mod.function_id == function_id:
            print(f"{Fore.MAGENTA}Modification ID: {mod.modification_id}{Style.RESET_ALL}")
            print(f"{Fore.MAGENTA}Modifier: {mod.modifier}{Style.RESET_ALL}")
            print(f"{Fore.MAGENTA}Description: {mod.description}{Style.RESET_ALL}")
            print(f"{Fore.MAGENTA}Modification Date: {mod.modification_date.isoformat()}{Style.RESET_ALL}")
            print()

    print(f"{Fore.YELLOW}Test Results:{Style.RESET_ALL}")
    for result in code_db.test_results:
        if result.function_id == function_id:
            print(f"{Fore.BLUE}Result ID: {result.result_id}{Style.RESET_ALL}")
            print(f"{Fore.BLUE}Test ID: {result.test_id}{Style.RESET_ALL}")
            print(f"{Fore.BLUE}Actual Result: {result.actual_result}{Style.RESET_ALL}")
            print(f"{Fore.BLUE}Status: {result.status.value}{Style.RESET_ALL}")
            print(f"{Fore.BLUE}Execution Date: {result.execution_date.isoformat()}{Style.RESET_ALL}")
            print()

In [18]:
def pretty_print_functions_table(code_db: CodeDatabase):
    # Collect data for the table
    table_data = []
    for func in code_db.functions.values():
        last_test_result = get_last_test_result(code_db, func.function_id)
        dot_color = get_dot_color(last_test_result)
        table_data.append([func.function_id, func.name, func.description, dot_color])

    # Define table headers
    headers = [f"{Fore.CYAN}Function ID{Style.RESET_ALL}", f"{Fore.CYAN}Name{Style.RESET_ALL}", f"{Fore.CYAN}Description{Style.RESET_ALL}", f"{Fore.CYAN}Status{Style.RESET_ALL}"]

    # Print the table
    print(tabulate(table_data, headers=headers, tablefmt="grid"))

In [19]:
def get_last_test_result(code_db: CodeDatabase, function_id: str) -> str:
    for result in code_db.test_results[::-1]:
        if result.function_id == function_id:
            return result.status.value
    return ""

In [20]:
def get_dot_color(status: str) -> str:
    if status == TestStatusEnum.PASSED.value:
        return f"{Fore.GREEN}✔{Style.RESET_ALL}"
    elif status == TestStatusEnum.FAILED.value:
        return f"{Fore.RED}✘{Style.RESET_ALL}"
    else:
        return ""

In [42]:
code_db = CodeDatabase()
print(code_db)

    # Step 1: Generate a Julia function


<CodeDatabase: AutomatedDevDB v1.0, Created: 2025-01-26T15:52:43.035764, Last Modified: 2025-01-26T15:52:43.035764, Functions: 0>
Generating Julia function for: implement a function 'longest_common_subsequence(str1::String, str2::String)' that finds 
    the longest common subsequence between two strings. A subsequence is a sequence that appears in the same 
    relative order but not necessarily contiguous. For example:
    - LCS of 'ABCDGH' and 'AEDFHR' is 'ADH'
    - LCS of 'AGGTAB' and 'GXTXAYB' is 'GTAB'
    The function should return the longest common subsequence as a string.


In [53]:
description = """Create a function, symbolic_derivative, that computes the symbolic derivative of a given univariate polynomial function (e.g., f(x) = 3x^2 + 2x + 1). Return the derivative as a function that can be evaluated at any point.

Challenge Areas:

Manipulating polynomials symbolically.
Translating mathematical rules into code.
Returning functions dynamically. """


In [54]:
print(f"Generating Julia function for: {description}")
generated_code_message = generate_julia_function(description)
generated_code = generated_code_message.parsed

Generating Julia function for: Create a function, symbolic_derivative, that computes the symbolic derivative of a given univariate polynomial function (e.g., f(x) = 3x^2 + 2x + 1). Return the derivative as a function that can be evaluated at any point.

Challenge Areas:

Manipulating polynomials symbolically.
Translating mathematical rules into code.
Returning functions dynamically. 


In [56]:
def format_code_block(code: str, language: str = "") -> str:
    """Format a code block with proper indentation and syntax highlighting."""
    return f"{Fore.YELLOW}```{language}\n{Style.RESET_ALL}{code}\n{Fore.YELLOW}```{Style.RESET_ALL}"



In [57]:
format_code_block(generated_code)

'\x1b```\n\x1bcode=\'function symbolic_derivative(poly::Vector{Int})\\n    derivative_coeffs = [(coeff * exp) for (exp, coeff) in enumerate(poly)][2:end]\\n    return (x) -> sum(c * x^(p-1) for (p, c) in enumerate(derivative_coeffs, start=2))\\nend\' tests="# Define a polynomial f(x) = 3x^2 + 2x + 1\\npolynomial = [1, 2, 3] # 1 + 2x + 3x^2, coefficients from constant to highest\\n\\n# Get the derivative function\\nf_prime = symbolic_derivative(polynomial)\\n\\n# Test the derivative function at x = 2\\n@assert f_prime(2) == 14 # f\'(x) = 6x + 2, so f\'(2) = 6(2) + 2 = 14" test_name=\'test_symbolic_derivative_at_specific_point\' test_description=\'Test the symbolic derivative of the polynomial 3x^2 + 2x + 1 at x=2 to ensure it returns 14.\' input_types=\'Vector{Int}\' return_types=\'Function\' short_description=\'Calculates the derivative of a simple polynomial function symbolically and returns it as a function.\' function_name=\'symbolic_derivative\'\n\x1b```\x1b'

In [58]:
generated_code = generated_code_message.parsed

In [59]:
generated_code

JuliaCodePackage(code='function symbolic_derivative(poly::Vector{Int})\n    derivative_coeffs = [(coeff * exp) for (exp, coeff) in enumerate(poly)][2:end]\n    return (x) -> sum(c * x^(p-1) for (p, c) in enumerate(derivative_coeffs, start=2))\nend', tests="# Define a polynomial f(x) = 3x^2 + 2x + 1\npolynomial = [1, 2, 3] # 1 + 2x + 3x^2, coefficients from constant to highest\n\n# Get the derivative function\nf_prime = symbolic_derivative(polynomial)\n\n# Test the derivative function at x = 2\n@assert f_prime(2) == 14 # f'(x) = 6x + 2, so f'(2) = 6(2) + 2 = 14", test_name='test_symbolic_derivative_at_specific_point', test_description='Test the symbolic derivative of the polynomial 3x^2 + 2x + 1 at x=2 to ensure it returns 14.', input_types='Vector{Int}', return_types='Function', short_description='Calculates the derivative of a simple polynomial function symbolically and returns it as a function.', function_name='symbolic_derivative')

In [60]:
 func = code_db.add_function(generated_code.function_name, generated_code.short_description, generated_code.code)

Added Function 374d36e4-9b87-401b-9d28-402cd317f6ae: symbolic_derivative


In [61]:
 # Step 3: Add unit tests for the function
code_db.add_unit_test(func.function_id, generated_code.test_name, generated_code.test_description, generated_code.tests)

Added UnitTest 9072d5cc-0a62-48c9-977e-3b2e789bbc53 to Function 374d36e4-9b87-401b-9d28-402cd317f6ae
Added UnitTest 9072d5cc-0a62-48c9-977e-3b2e789bbc53 to Function 374d36e4-9b87-401b-9d28-402cd317f6ae


<UnitTest 9072d5cc-0a62-48c9-977e-3b2e789bbc53: test_symbolic_derivative_at_specific_point for Function 374d36e4-9b87-401b-9d28-402cd317f6ae>

In [62]:
formatted_code = format_code_block(generated_code.code, language="julia")
print(formatted_code)

```julia
function symbolic_derivative(poly::Vector{Int})
    derivative_coeffs = [(coeff * exp) for (exp, coeff) in enumerate(poly)][2:end]
    return (x) -> sum(c * x^(p-1) for (p, c) in enumerate(derivative_coeffs, start=2))
end
```


In [63]:
max_attempts = 5
attempt = 0
while attempt < max_attempts:
    print(f"\nAttempt {attempt + 1} of {max_attempts}")
    
    # Execute tests
    print("Executing Tests...")
    test_results = code_db.execute_tests()
    
    # Check if any tests failed
    failed_tests = [r for r in test_results if r.status == TestStatusEnum.FAILED]
    
    if not failed_tests:
        print("All tests passed!")
        break
        
    print(f"Failed tests: {len(failed_tests)}")
    for result in failed_tests:
        print(f"Test {result.test_id} failed: {result.actual_result}")
    
    # Ask AI to fix the code
    fix_description = f"Fix the function. Current tests failed with: {format_code_block([r.actual_result for r in failed_tests])}"
    print(f"\nRequesting fix: {fix_description}")
    fixed_code_message = modify_julia_function(fix_description)
    fixed_code = fixed_code_message.parsed.code
    
    # Update function with fix
    code_db.modify_function(func.function_id, "AI", f"Fix attempt {attempt + 1}", fixed_code)
    
    attempt += 1

if attempt == max_attempts:
    print("\nMaximum fix attempts reached without success")
else:
    print(f"\nFixed in {attempt + 1} attempts")


Attempt 1 of 5
Executing Tests...
Executing all unit tests...
Running Test 'test_process_data_various_inputs' for Function ID d031fddc-715c-49ec-903d-3c7c39319262
Test Result: <TestResult fde242c9-8584-4ddf-82f4-b0e1a1ebf39c: Test aa0075ce-f65f-472e-a313-b08430a28773 for Function d031fddc-715c-49ec-903d-3c7c39319262 Status: Passed>
Running Test 'test_symbolic_derivative_at_specific_point' for Function ID 374d36e4-9b87-401b-9d28-402cd317f6ae
Test Failed: ERROR: LoadError: MethodError: no method matching enumerate(::Vector{Int64}; start::Int64)

Closest candidates are:
  enumerate(::Any) got unsupported keyword argument "start"
   @ Base iterators.jl:200

Stacktrace:
 [1] (::var"#2#5"{Vector{Int64}})(x::Int64)
   @ Main C:\Users\GGPC\AppData\Local\Temp\tmpykd03mg1.jl:4
 [2] top-level scope
   @ C:\Users\GGPC\AppData\Local\Temp\tmpykd03mg1.jl:14
in expression starting at C:\Users\GGPC\AppData\Local\Temp\tmpykd03mg1.jl:14
Test Result: <TestResult 543d42d4-7ff5-4876-ad19-9768e069b0cd: Test

TypeError: modify_julia_function() missing 1 required positional argument: 'function_code'

In [64]:
fix_description

'Fix the function. Current tests failed with: \x1b```\n\x1b[\'ERROR: LoadError: MethodError: no method matching enumerate(::Vector{Int64}; start::Int64)\\n\\nClosest candidates are:\\n  enumerate(::Any) got unsupported keyword argument "start"\\n   @ Base iterators.jl:200\\n\\nStacktrace:\\n [1] (::var"#2#5"{Vector{Int64}})(x::Int64)\\n   @ Main C:\\\\Users\\\\GGPC\\\\AppData\\\\Local\\\\Temp\\\\tmpykd03mg1.jl:4\\n [2] top-level scope\\n   @ C:\\\\Users\\\\GGPC\\\\AppData\\\\Local\\\\Temp\\\\tmpykd03mg1.jl:14\\nin expression starting at C:\\\\Users\\\\GGPC\\\\AppData\\\\Local\\\\Temp\\\\tmpykd03mg1.jl:14\']\n\x1b```\x1b'

# Specification: CLI Interface for `code_db.py`

## Overview
This document specifies the requirements and design for a Command-Line Interface (CLI) to interact with `code_db.py`, a Python module for managing a code database (functions, tests, results, and modifications).

## Requirements
- The CLI must allow users to add, modify, list, and query functions in the code database.
- Users must be able to add and run unit tests, view test results, and track modifications.
- The CLI should provide clear help messages and error handling.
- All operations should be accessible via subcommands and options.

## Command Structure

```
code_db_cli.py <command> [options]
```

### Core Commands

- `add-function`  
  Add a new function to the database.
  - Options: `--name`, `--description`, `--code-file`
- `list-functions`  
  List all functions with summary info.
- `show-function`  
  Show details for a specific function.
  - Options: `--id`
- `modify-function`  
  Modify an existing function.
  - Options: `--id`, `--modifier`, `--description`, `--code-file`
- `add-test`  
  Add a unit test to a function.
  - Options: `--function-id`, `--name`, `--description`, `--test-file`
- `run-tests`  
  Run all or specific tests.
  - Options: `--function-id` (optional)
- `show-results`  
  Show test results.
  - Options: `--function-id` (optional)
- `list-modifications`  
  List modifications for a function.
  - Options: `--function-id`

### Global Options

- `-h`, `--help`  
  Show help message.

## Example Usage

```sh
python code_db_cli.py add-function --name "symbolic_derivative" --description "Computes symbolic derivative..." --code-file symbolic_derivative.jl
python code_db_cli.py list-functions
python code_db_cli.py show-function --id FUNC123
python code_db_cli.py add-test --function-id FUNC123 --name "test_derivative" --description "Test for derivative" --test-file test_derivative.jl
python code_db_cli.py run-tests --function-id FUNC123
python code_db_cli.py show-results --function-id FUNC123
```

## Extensibility Notes

- The CLI should be implemented using Python's `argparse` or `click` for easy extension.
- New commands (e.g., export, import, search) can be added as subcommands.
- The CLI should be modular, with each command mapped to a function in `code_db.py`.
- Support for additional languages or test frameworks can be added via plugins or configuration.